In [ ]:
import numpy as np
import numpy.random as nr
import matplotlib

# matplotlib.use('Qt5Agg')
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
#from Super_function import *
import scipy.signal as sps

from scipy import ndimage
import multiprocessing
from multiprocessing import Pool, TimeoutError

def super_surface(a0=1., b0=1., c0=1., mm=1., nn=1., res=50):
    Epl1 = (mm - 1) * 1 / 4 + 0.15
    Epl2 = (nn - 1) * 1 / 4 + 0.15
    
    eta, phi = np.mgrid[0:2 * np.pi:res * 1j, 0:np.pi:res * 1j]

    x00 = a0 * abs(np.cos(eta)) ** Epl1 * abs(np.cos(phi)) ** Epl2 * np.sign(np.cos(phi) * np.cos(eta))
    y00 = b0 * abs(np.cos(eta)) ** Epl1 * abs(np.sin(phi)) ** Epl2 * np.sign(np.sin(phi) * np.cos(eta))
    z00 = c0 * abs(np.sin(eta)) ** Epl1 * np.sign(np.sin(eta))

    return x00, y00, z00

def super_volume(x, y, z, a, b, c, mm, nn):
    Epl1 = (mm - 1) * 1 / 4 + 0.15
    Epl2 = (nn - 1) * 1 / 4 + 0.15

    rr = ((np.abs(x / a)) ** (2 / Epl1) + (np.abs(y / b)) ** (2 / Epl1)) ** (Epl1 / Epl2) + (np.abs(z / c)) ** (
            2 / Epl2)
    return rr

def rsgeng_3D(N, rL, h, clx, cly, clz):
    np.random.seed()
    ZZ = h * nr.randn(N, N, N)

    x, y, z = np.linspace(-rL / 2, rL / 2, num=N, endpoint=True), \
              np.linspace(-rL / 2, rL / 2, num=N, endpoint=True), \
              np.linspace(-rL / 2, rL / 2, num=N, endpoint=True)

    [X, Y, Z] = np.meshgrid(x, y, z, indexing='ij')
    F = np.exp(-X ** 2 / ((clx ** 2) / 2) - Y ** 2 / ((cly ** 2) / 2) - Z ** 2 / ((clz ** 2) / 2))
    f = 2 / np.sqrt(np.pi) * (rL / N / np.sqrt(clx * cly * clz)) * np.fft.ifftn(np.fft.fftn(ZZ) * np.fft.fftn(F))
    return f, x, y, z

global cov_k

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

c_kx, c_ky, c_kz = np.mgrid[-1:1:3j, -1:1:3j, -1:1:3j]
cov_k = gaussian(c_kx, 0, 2) * gaussian(c_ky, 0, 2) * gaussian(c_kz, 0, 2)


def main(number=100):

    nnn = int(number)
    num_num = range(0, nnn)

    arg = zip(num_num)

    real_amp_2d = np.zeros((nnn, 32, 32))
    real_pha_2d = np.zeros((nnn, 32, 32))

    n_cpu = cpu(dif=True)
    pool = Pool(processes=n_cpu)  
    
    ii = 0
    for real_amp_2dd, real_pha_2dd, real_space_complex_obj, reci_space_complex_obj_2D, real_space_complex_obj_2D in pool.imap(multi_run_wrapper, arg):
        
        real_amp_2d[ii] = real_amp_2dd
        real_pha_2d[ii] = real_pha_2dd
         
        ii += 1

    pool.terminate()

    np.save('real_amp_2d.npy', real_amp_2d)
    np.save('real_pha_2d.npy', real_pha_2d)

    reci_space_amp = np.abs(np.fft.fftshift(np.fft.fftn(real_space_complex_obj)))

    fig, ax = plt.subplots(2, 2)

    # print((reci_space_amp[64, :, :]))

    ax[0, 0].imshow(np.log10(np.squeeze((reci_space_amp[32, :, :]))),
                    vmin=np.max(np.log10(np.squeeze((reci_space_amp[32, :, :])))) - 4,
                    vmax=np.max(np.log10(np.squeeze((reci_space_amp[32, :, :])))))

    ax[0, 1].imshow(np.log10(np.abs(reci_space_complex_obj_2D)),
                    vmin=np.max(np.log10(np.abs(reci_space_complex_obj_2D))) - 4,
                    vmax=np.max(np.log10(np.abs(reci_space_complex_obj_2D))))

    ax[1, 0].imshow(np.abs(real_space_complex_obj_2D))

    ax[1, 1].imshow(np.angle(real_space_complex_obj_2D))

    plt.show(block=True)


def cpu(dif=False):
    n_cpu = 1
    if dif:
        print(f"The core of cup is {n_cpu:02}")
        
    return n_cpu


def multi_run_wrapper(args):
    return single_particle()


def single_particle():
    # print(nn)

    m1, n1 = 1 + np.random.rand(1) * 9, 1 + np.random.rand(1) * 9

    a1, b1, c1 = 5+np.random.rand(1)*5, 5+np.random.rand(1)*5, 5+np.random.rand(1)*5

    xx0, yy0, zz0 = np.mgrid[-32:32:64j, -32:32:64j, -32:32:64j]

    # u, v = np.random.rand(1), np.random.rand(1)
    theta, phi = 2 * np.pi * np.random.rand(1), np.random.rand(1)

    r_tp = np.array(([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)],
                     [np.cos(theta) * np.cos(phi), np.cos(theta) * np.sin(phi), -np.sin(theta)],
                     [-np.sin(phi), np.cos(phi), 0]))

    for ii in range(64):
        for jj in range(64):
            for kk in range(64):
                x, y, z = np.dot(r_tp, np.array((xx0[ii, jj, kk], yy0[ii, jj, kk], zz0[ii, jj, kk])))

                xx0[ii, jj, kk], yy0[ii, jj, kk], zz0[ii, jj, kk] = x, y, z

    r0 = super_volume(xx0, yy0, zz0, a1, b1, c1, m1, n1)

    real_space_amp = np.zeros((64, 64, 64))
    real_space_amp[r0 <= 1] = 1.

    real_space_amp = sps.fftconvolve(sps.fftconvolve(real_space_amp, cov_k, mode='same'), cov_k, mode='same')

    real_space_amp /= np.max(real_space_amp)
    real_space_amp = real_space_amp * (real_space_amp > 0.05)

    N = 64
    rL = 64
    h = 3.5*np.random.rand(1)
    clx = 10 + np.random.rand(1)*5
    cly = 10 + np.random.rand(1)*5
    clz = 10 + np.random.rand(1)*5

    real_space_phase, _, _, _ = rsgeng_3D(N, rL, h, clx, cly, clz)

    real_space_phase = np.abs(real_space_phase)

    real_space_phase = np.angle(np.exp(1j * (real_space_phase - np.mean(real_space_phase[real_space_amp < 0.05]))))

    real_space_phase[real_space_amp < 0.05] = 0

    real_space_complex_obj = real_space_amp * np.exp(1j * real_space_phase)

    real_space_complex_obj_2D = np.sum(real_space_complex_obj, axis=0)
    real_space_complex_obj_2D /= np.max(np.abs(real_space_complex_obj_2D))

    reci_space_complex_obj_2D = np.fft.fftshift(np.fft.fftn(real_space_complex_obj_2D))

    real_amp_2d = np.abs(real_space_complex_obj_2D)[(32 - 16):(32 + 16), (32 - 16):(32 + 16)]
    real_pha_2d = np.angle(real_space_complex_obj_2D)[(32 - 16):(32 + 16), (32 - 16):(32 + 16)]

    return real_amp_2d, real_pha_2d, real_space_complex_obj, reci_space_complex_obj_2D, real_space_complex_obj_2D


if __name__ == "__main__":
    main()

    
# there will be two file generated, after run this script.
# real_amp_2d.npy is the amplitude of the particle
# real_pha_2d.npy is the phase of the particle

# number of data to generate
number = 1e2

# generate the data
main(number)

print('Finished')

The core of cup is 01
